In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim


import torchvision
from torchvision import transforms, datasets
from torchsummary import summary #just for debugging
from torch.utils.tensorboard import SummaryWriter


from tqdm import tqdm
import copy
import time
import os

## Parameters

In [3]:
TRAIN_DIR = "./Data/seg_train"
TEST_DIR = "./Data/seg_test"

genre_dict = {"buildings":0,"forest":1,"glacier":2,"mountain":3,"sea":4,"street":5}

BATCH_SIZE = 32
IMAGE_SIZE = (64,64) # resize the images
SEED = 128
VALIDATION_SPLIT = 0.2

EPOCHS = 50

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Load Data

In [5]:
def Dataloader(traindir, testdir, image_size, batch_size, validation_split):
    
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
    
    train_transform = transforms.Compose([transforms.Resize(image_size),
                                          transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0),
                                          transforms.RandomVerticalFlip(0.5),
                                          transforms.ToTensor(),
                                          normalize])
    
    test_transform = transforms.Compose([transforms.Resize(image_size),
                                         transforms.ToTensor(),
                                         normalize])
    
    
    train_data = datasets.ImageFolder(traindir, transform = train_transform)
    test_data = datasets.ImageFolder(testdir, transform = test_transform)
    
    
    classes = train_data.classes
    

    length_val_ds = int(len(train_data)*validation_split)
    length_train_ds = len(train_data) - length_val_ds
    
    train_data, valid_data = torch.utils.data.random_split(train_data, [length_train_ds, length_val_ds])
    
    
    trainloader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size = 6)
    validloader = torch.utils.data.DataLoader(valid_data, shuffle = True, batch_size = BATCH_SIZE)
    testloader = torch.utils.data.DataLoader(test_data, shuffle = True, batch_size = BATCH_SIZE)
    
    return trainloader, validloader, testloader, classes

In [6]:
train_ds, val_ds, test_ds, class_names = Dataloader(traindir=TRAIN_DIR, testdir=TEST_DIR, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE, validation_split=VALIDATION_SPLIT)
class_names

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

## Show Images

In [ ]:
generator = iter(train_ds)
                    
labels = [0, 1, 2, 3, 4, 5]
counter = 0
images = []
                    
while True:
    inputs, classes = next(generator)
    classes = classes.numpy()
    
    if counter in classes:
        index = np.where(classes == counter)[0][0]
        images.append(inputs[index])
        counter += 1
    
    if counter == 6:
        break

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    #mean = np.array([0.485, 0.456, 0.406])
    #std = np.array([0.229, 0.224, 0.225])
    #inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.figure(figsize = (15,10))
    plt.imshow(inp, aspect='auto')
    if title is not None:
        plt.title(title)
    plt.savefig('example_images.png')
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(train_ds))

# Make a grid from batch
out = torchvision.utils.make_grid(images, nrow = 3)

#imshow(out, title=[class_names[x] for x in classes])
imshow(out, title=["buildings","forest","glacier","mountain","sea","street"])

## Model

In [7]:
#######################################################
#######################################################
#################### NETWORK SMALL ####################
#######################################################
#######################################################
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
                   
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, stride = 1, padding = 0)
        self.relu_conv_1 = nn.ReLU()
        self.Batch_Norm2D_1 = nn.BatchNorm2d(32)
        self.maxpool_1 = nn.MaxPool2d(kernel_size = (2,2), stride = 2)
        
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 1, padding = 0)
        self.relu_conv_2 = nn.ReLU()
        #self.drop2d_1 = nn.Dropout2d(p=0.2)
        #self.Batch_Norm2D_2 = nn.BatchNorm2d(32)
        self.maxpool_2 = nn.MaxPool2d(kernel_size = (2,2), stride = 2)
        
        self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 16, kernel_size = 3, stride = 1, padding = 0)
        self.relu_conv_3 = nn.ReLU()
        self.Batch_Norm2D_3 = nn.BatchNorm2d(16)
        self.maxpool_3 = nn.MaxPool2d(kernel_size = (2,2), stride = 2)

        
        #in_channels = 10
        #out_channels = 32
        #self.depth_conv_1 = Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=3, groups=10)
        #self.point_conv_1 = Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1)
        
        
        self.flatten = nn.Flatten()
        
        #self.Linear1 = nn.LazyLinear(12)
        #self.relu1 = nn.ReLU()
        #self.drop1 = nn.Dropout(0.5)
        #self.BatchNorm1 = nn.LazyBatchNorm1d()
        
        #self.Linear2 = nn.LazyLinear(12)
        #self.relu2 = nn.ReLU()
        
        self.Linear3 = nn.LazyLinear(6)
        self.softmax = nn.Softmax(dim = 1)
        
        
        
    def forward(self, x):

        x = self.conv1(x)
        x = self.relu_conv_1(x)
        x = self.Batch_Norm2D_1(x)
        x = self.maxpool_1(x)

        x = self.conv2(x)
        x = self.relu_conv_2(x)
        #x = self.drop2d_1(x)
        #x = self.Batch_Norm2D_2(x)
        x = self.maxpool_2(x)
        
        x = self.conv3(x)
        x = self.relu_conv_3(x)
        x = self.Batch_Norm2D_3(x)
        x = self.maxpool_3(x)
        
        
        x = self.flatten(x)
        
        #x = self.Linear1(x)
        #x = self.relu1(x)
        #x = self.drop1(x)
        #x = self.BatchNorm1(x)
        
        x = self.Linear3(x)
        x = self.softmax(x)

        return x
    
network = Net()
optimizer = optim.Adam(network.parameters())
summary(network, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 30, 30]             896
              ReLU-2           [-1, 32, 30, 30]               0
       BatchNorm2d-3           [-1, 32, 30, 30]              64
         MaxPool2d-4           [-1, 32, 15, 15]               0
            Conv2d-5           [-1, 32, 13, 13]           9,248
              ReLU-6           [-1, 32, 13, 13]               0
         MaxPool2d-7             [-1, 32, 6, 6]               0
            Conv2d-8             [-1, 16, 4, 4]           4,624
              ReLU-9             [-1, 16, 4, 4]               0
      BatchNorm2d-10             [-1, 16, 4, 4]              32
        MaxPool2d-11             [-1, 16, 2, 2]               0
          Flatten-12                   [-1, 64]               0
           Linear-13                    [-1, 6]             390
          Softmax-14                   

/Users/vinz/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [8]:
#######################################################
#######################################################
#################### NETWORK LARGE ####################
#######################################################
#######################################################
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
                   
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, stride = 1, padding = 0)
        self.relu_conv_1 = nn.ReLU()
        self.Batch_Norm2D_1 = nn.BatchNorm2d(32)
        
        
        self.maxpool_2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 1)
        self.relu_conv_2 = nn.ReLU()
        self.Batch_Norm2D_2 = nn.BatchNorm2d(64)
        
        
        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 16, kernel_size = 3, stride = 1, padding = 1)
        self.relu_conv_3 = nn.ReLU()
        
        self.conv5 = nn.Conv2d(in_channels = 64, out_channels = 16, kernel_size = 1, stride = 1, padding = 0)
        self.relu_conv_5 = nn.ReLU()
        
        
        self.maxpool_6 = nn.MaxPool2d(kernel_size = (2,2), stride = 2)
        self.conv6 = nn.Conv2d(in_channels = 32, out_channels = 16, kernel_size = 3, stride = 1, padding = 0)
        self.relu_conv_6 = nn.ReLU()
        self.Batch_Norm2D_6 = nn.BatchNorm2d(16)
        

        self.conv7 = nn.Conv2d(in_channels = 16, out_channels = 8, kernel_size = 3, stride = 1, padding = 0)
        self.relu_conv_7 = nn.ReLU()
        
        
        
        self.flatten = nn.Flatten()
        
        self.Linear1 = nn.LazyLinear(32)
        self.relu1 = nn.ReLU()
        self.drop1 = nn.Dropout(0.5)
        
        self.Linear3 = nn.LazyLinear(6)
        self.softmax = nn.Softmax(dim = 1)
        
        
        
    def forward(self, x):

        x = self.conv1(x)
        x = self.relu_conv_1(x)
        x = self.Batch_Norm2D_1(x)
        
        x = self.maxpool_2(x)
        x = self.conv2(x)
        x = self.relu_conv_2(x)
        x = self.Batch_Norm2D_2(x)
        
        x1 = self.conv3(x)
        x1 = self.relu_conv_3(x1)
        
        x2 = self.conv5(x)
        x2 = self.relu_conv_5(x2)
        
        x = torch.cat((x1,x2),1)
        
        x = self.maxpool_6(x)
        x = self.conv6(x)
        x = self.relu_conv_6(x)
        x = self.Batch_Norm2D_6(x)
        
        x = self.conv7(x)
        x = self.relu_conv_7(x)
        
        
        x = self.flatten(x)
        
        x = self.Linear1(x)
        x = self.relu1(x)
        x = self.drop1(x)
        
        x = self.Linear3(x)
        x = self.softmax(x)

        return x
    
network = Net()
optimizer = optim.Adam(network.parameters())
summary(network, input_size=(3, 64, 64))


#### CURRENTTTT

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 62, 62]             896
              ReLU-2           [-1, 32, 62, 62]               0
       BatchNorm2d-3           [-1, 32, 62, 62]              64
         MaxPool2d-4           [-1, 32, 31, 31]               0
            Conv2d-5           [-1, 64, 31, 31]          18,496
              ReLU-6           [-1, 64, 31, 31]               0
       BatchNorm2d-7           [-1, 64, 31, 31]             128
            Conv2d-8           [-1, 16, 31, 31]           9,232
              ReLU-9           [-1, 16, 31, 31]               0
           Conv2d-10           [-1, 16, 31, 31]           1,040
             ReLU-11           [-1, 16, 31, 31]               0
        MaxPool2d-12           [-1, 32, 15, 15]               0
           Conv2d-13           [-1, 16, 13, 13]           4,624
             ReLU-14           [-1, 16,

In [187]:
batch, label = next(iter(test_ds))
batch_np = batch.to(device)
yhat = network(batch_np) # Give dummy batch to forward().

input_names = ['Input']
output_names = ['Output']
torch.onnx.export(network, batch_np, 'Network.onnx', input_names=input_names, output_names=output_names)

#### tb = SummaryWriter()
images, labels = next(iter(train_ds))
grid = torchvision.utils.make_grid(images)
tb.add_image("images", grid)
tb.add_graph(network, images)
tb.close()

In [9]:
def train_model(model, dataloader_train, dataloader_val, lambdas, criterion, optimizer, scheduler, num_epochs=25, Fine_Tuning = False):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    lambda_1, lambda_2 = lambdas
    
    writer = SummaryWriter()

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        if Fine_Tuning and (epoch > 0.9*num_epochs):
            for param in network.transfer.parameters():
                param.requires_grad = True

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                dataloader = dataloader_train
            else:
                model.eval()   # Set model to evaluate mode
                dataloader = dataloader_val

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloader, total = len(dataloader)):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    
                    # add l1 and l2 regularization
                    #l1_regularization, l2_regularization = 0,0
                    #for param in model.parameters():
                    #    l1_regularization += lambda_1*torch.norm(param, 1)#**2
                    #    l2_regularization += lambda_2*torch.norm(param, 2)**2
                        
                    loss = criterion(outputs, labels) #+ l1_regularization + l2_regularization

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

                
            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)
            
            if phase == "train":
                writer.add_scalar("Loss/train", epoch_loss, epoch)
                writer.add_scalar("Accuracy/train", epoch_acc, epoch)
            else:         
                writer.add_scalar("Loss/test", epoch_loss, epoch)
                writer.add_scalar("Accuracy/test", epoch_acc, epoch)
            
            for name, weight in model.named_parameters():
                tb.add_histogram(name,weight, epoch)
                tb.add_histogram(f'{name}.grad',weight.grad, epoch)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
                Save_path = f"./Pytorch_Model/checkpoints/EPOCH_{epoch}_ACC:_{epoch_acc}_%"
                torch.save(model.state_dict(), Save_path)
                

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    writer.flush()
    writer.close()
    return model, writer

In [10]:
network = network.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(network.parameters(), lr=0.001)#, weight_decay=1e-3)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=25, gamma=0.1)

lambdas = (1e-5, 1e-5)
lambdas = (1e-5, 1e-5) # NOT USED

# If Fine_Tuning = True, the whole network (including pretrained one)
# will be trained in the last 10% of the total epochs
Fine_Tuning = True

network, history = train_model(network, train_ds, val_ds, lambdas, criterion, optimizer_ft, exp_lr_scheduler,
                           num_epochs = 100, Fine_Tuning = False)

Epoch 0/99
----------


100%|█████████████████████████████████████████| 351/351 [02:50<00:00,  2.06it/s]


train Loss: 1.5499 Acc: 0.5360


100%|███████████████████████████████████████████| 88/88 [00:30<00:00,  2.86it/s]


val Loss: 1.4625 Acc: 0.5984

Epoch 1/99
----------


...

train Loss: 1.2489 Acc: 0.7956


100%|███████████████████████████████████████████| 88/88 [01:15<00:00,  1.17it/s]


val Loss: 1.2517 Acc: 0.7904

Epoch 32/99
----------


 11%|████▊                                     | 40/351 [00:24<03:11,  1.62it/s]


KeyboardInterrupt: 

## Testing

In [ ]:
def validate(dataset):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in tqdm(dataset, total = len(dataset)):
            images, labels = data
            # calculate outputs by running images through the network
            outputs = network(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')
    return 100 * correct // total

In [ ]:
accuracy = validate(test_ds)

In [ ]:
def per_class_validate(dataset):
    classes = dataset.dataset.classes
    # prepare to count predictions for each class
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}
    
    # again no gradients needed
    with torch.no_grad():
        for data in tqdm(dataset, total = len(dataset)):
            images, labels = data
            outputs = network(images)
            _, predictions = torch.max(outputs, 1)
            # collect the correct predictions for each class
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1
    
    
    # print accuracy for each class
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

In [ ]:
per_class_validate(test_ds)

## Saving and Loading Model

In [ ]:
from datetime import datetime

# set to true if want to save model
dt_string = datetime.now().isoformat()

Save_path = f"./Pytorch_Model/ACC:_{accuracy}_%__{dt_string}"
torch.save(network.state_dict(), Save_path)

In [15]:
# Loading Model
#model = Net()
#model.load_state_dict(torch.load(Save_path))

<All keys matched successfully>

## Plotting

In [ ]:
from matplotlib import pyplot as plt

class_names = train_ds.dataset.dataset.classes


def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


def visualize_model(model, num_images=8):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(val_ds):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]} \n actual: {class_names[labels[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
visualize_model(network)